# GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers

En el paper [GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323) se expone la necesidad de crear un método de cuantización post entrenamiento que no degrade la calidad del modelo. En este post hemos visto el método [llm.int8()](https://maximofn.com/llm-int8/) que cuantiza a INT8 algunos vectores de las matrices de pesos, siempre y cuando ninguno de sus valores sobrepase un valor umbral, lo cual está muy bien, pero no cuantizan todos los pesos del modelo. En este paper proponen un método que cuantiza todos los pesos del modelo a 4 y 3 bits, sin degradar la calidad del modelo. Lo que supone un ahorro considerable de memoria, no solo porque se cuantizan todos los pesos, sino porque además se hace a 4, 3 bits (y hasta 1 y 2 bits en ciertas condiciones), en vez de a 8 bits.

## Trabajos en los que se basa

### Cuantización por capas

Por un lado se basan en los trabajos `Nagel et al., 2020`; `Wang et al., 2020`; `Hubara et al., 2021` y `Frantar et al., 2022`, que proponen cuantizar los pesos de las capas de una red neuronal a 4 y 3 bits, sin degradar la calidad del modelo.

Teniendo un conjunto de datos `m` proveniente de un dataset, a cada capa `l` se le meten los datos y se obtiene la salida de los pesos `W` de dicha capa. Así que lo que se hace es buscar unos pesos nuevos `Ŵ` cuantizados que minimicen el error cuadrático en relación con la salida de la capa de precisión total

`argmin_Ŵ||WX− ŴX||^2`

Los valores de `Ŵ` se establecen antes de realizar el proceso de cuantización y durante el proceso, cada parámetro de `Ŵ` puede cambiar de valor independientemente sin depender del valor de los demás parámetros de `Ŵ`.

### Optimal brain quantization (OBQ)

En el trabajo de `OBQ` de `Frantar et al., 2022` optimizan el proceso de cuantización por capas anterior, haciendo que llegue a ser hasta 3 veces más rápido. Esto ayuda con los modelos grandes, ya que cuantizar un modelo grande puede llevar mucho tiempo.

El método `OBQ` es un enfoque para resolver el problema de cuantificación por capas en modelos de lenguaje. `OBQ` parte de la idea de que el error cuadrático se puede descomponer en la suma de errores individuales para cada fila de la matriz de pesos. Luego, el método cuantifica cada peso de manera independiente, actualizando siempre los pesos no cuantificados para compensar el error incurrido por la cuantificación.

El método es capaz de cuantificar modelos de tamaño mediano en tiempos razonables, pero como es un algoritmo de complejidad cúbica hace que sea extremadamente costoso aplicarlo a modelos con miles de millones de parámetros.

## Algoritmo de GPTQ

### Paso 1: Información de orden arbitrario

En `OBQ` se buscaba la fila de pesos que creaba menor error cuadrático medio para cuantizar, pero se dieron cuenta que al hacerlo de manera aleatoria no aumentaba mucho el error cuadrático medio final. Por lo que en vez de buscar la fila que minimiza el error cuadrático medio, que creaba una complejidad cúbica en el algoritmo, se hace siempre en el mismo orden. Gracias a esto se reduce mucho el tiempo de ejecución del algoritmo de cuantización.

### Paso 2: Actualizaciones lazy batch

Como la actualización de los pesos se hace fila a fila, hace que sea un proceso lento y no se aproveche del todo el hardware. Por lo que proponen realizar las actualizaciones en lotes de `B=128` filas. De esta manera se aprovecha mejor el hardware y se reduce el tiempo de ejecución del algoritmo.

### Paso 3: Reformulación de Cholesky

El problema de hacer las actualizaciones por lotes es que, debido a la gran escala de los modelos, se pueden producir errores numéricos que afectan la precisión del algoritmo. Concretamente, se pueden obtener matrices indefinidas, lo que provoca que el algoritmo actualice los pesos restantes en direcciones incorrectas, lo que resulta en una cuantización muy mala.

Para solucionar esto, los autores del paper proponen utilizar una reformulación de Cholesky, que es un método más numéricamente estable.

## Resultados de GPTQ

A continuación se muestran dos gráficas con la medida de la perplejidad (perplexity) en el dataset `WikiText2` para todos los tamaños de los modelos OPT y BLOOM. Se puede ver que con la técnica de cuantización RTN, la perplejidad en algunos tamaños aumenta mucho, mientras que con GPTQ se mantiene similar a la que se obtinene con el modelo en FP16

![GPTQ-figure1](https://maximofn.com/wp-content/uploads/2024/07/GPTQ-figure1.webp)

A continuación se muestran otras gráficas, pero con la medida de del accuracy en el dataset `LAMBADA`. Ocurre lo mismo, mientras que GPTQ se mantiene similar a lo obtenido con FP16, otros métodos de cuantización degradan mucho la calidad del modelo

![GPTQ-figure3](https://maximofn.com/wp-content/uploads/2024/07/GPTQ-figure3.webp)

## Cuantización extrema

En las gráficas anteriores se han mostrado los resultados de cuantizar el modelo a 3 y 4 bits, pero podemos cuantizarlos a 2bits, e incluso a 1 solo bit.

Modificando el tamaño de los batches al utilizar el algoritmo podemos obtener buenos resultados cuantizando tanto el modelo

|Modelo | FP16 | g128 | g64 | g32 | 3 bits|
|-------|-------|--------|--------|--------|---------|
|OPT-175B | 8.34 | 9.58 | 9.18 | 8.94 | 8.68|
|BLOOM | 8.11 | 9.55 | 9.17 | 8.83 | 8.64|

En la tabla anterior se puede ver el resultado de la perplejidad en el dataset `WikiText2` para los modelos `OPT-175B` y `BLOOM` cuantizados a 3 bits. Se puede ver que a medida que se usen batches más pequeños, la perplejidad disminuye, lo que significa que la calidad del modelo cuantizado es mejor. Pero tiene el problema de que el algoritmo tarda más en ejecutarse

## Descuantificación dinámica en la inferencia

Durante la inferencia se raliza algo llamado `descuantificación dinámica` (`dynamic dequantization`) para poder realizar la inferencia. Se va descuantificando cada capa a medida que se va pasando por ellas.

Para ello desarrollaron un kernel de que descuantifica las matrices y realiza los productos matriciales. Si bien la descuantificación consume más cálculos, el kernel tiene que acceder a mucha menos memoria, lo que genera aceleraciones significativas

## Velocidad de inferencia

Los autores del paper realizaron una prueba cuantizando el modelo BLOOM-175B a 3 bits, lo que ocupaba unos 63 GB de memoria VRAM, incluidos los embeddings y la capa de salida que se mantienen en FP16. Además mantener la ventana de contexto de 2048 tokens consume unos 9 GB de memoria, lo que hace en total unos 72 GB de memoria VRAM. Cuantizaron en 3 bits y no en 4 para poder realizar este experimento y poder meter el modelo en una sola GPU Nvidia A100 de 80 GB de memoria VRAM.

Para comparar, la inferencia normal en FP16 requiere unos 350 GB de memoria VRAM, lo que equivale a 5 GPUs Nvidia A100 de 80 GB de memoria VRAM. Y la inferencia cuantizando en 8 bits mediante [llm.int8()](https://maximofn.com/llm-int8/) requiere 3 de dichas GPUs.

A continuación, se muestra una tabla con la inferencia del modelo en FP16 y cuantizado a 3 bits e GPUs Nvidia A100 de 80 GB de memoria VRAM y Nvidia A6000 de 48 GB de memoria VRAM.

|GPU (VRAM)| tiempo promedio por token en FP16 (ms) | tiempo promedio por token en 3 bit (ms) | Aceleración | Reducción de GPUs necesarias |
|-------|---------|--------|-------------|------------------|
|A6000 (48GB) | 589 | 130 | ×4.53 | 8→ 2|
|A100 (80GB) | 230 | 71 | ×3.24 | 5→ 1|

Por ejemplo, utilizando los kernels, el modelo OPT-175B de 3 bits se ejecuta en una sola A100 (en vez de en 5) y es aproximadamente 3,25 veces más rápido que la versión FP16 en términos de tiempo promedio por token.

La GPU NVIDIA A6000 tiene un ancho de banda de memoria mucho menor, por lo que esta estrategia es aún más efectiva: ejecutar el modelo OPT-175B de 3 bits en 2 GPUs A6000 (en vez de en 8) es aproximádamente 4.53 veces más rápido que la versión FP16.